In [1]:
#modified version on our dataset

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
import pandas as pd
import numpy as np
import re

#needed for text processing and analytics
import nltk
import nltk.stem

from nltk.tokenize import word_tokenize, sent_tokenize, PunktSentenceTokenizer
from nltk import pos_tag, sent_tokenize, word_tokenize, BigramAssocMeasures,\
    BigramCollocationFinder, TrigramAssocMeasures, TrigramCollocationFinder
from nltk.corpus import stopwords
from string import punctuation

'\nfrom plotly import tools\nimport plotly as py\nfrom plotly.offline import init_notebook_mode, iplot\ninit_notebook_mode(connected=True)\nimport plotly.graph_objs as go\n\nimport sys\nimport warnings\n\nif not sys.warnoptions:\n    warnings.simplefilter("ignore")\n\n\n#to plot inside the document\n%matplotlib inline\nimport matplotlib.pyplot as plt\n'

In [2]:
#load the dataset-our version-from consolidated-transcripts
#import data
df = pd.read_csv('D:/Download/KUL/2nd semester/Modern Data Analysis/project/consolidated-transcripts.csv')
print('We have',len(df), 'speeches in the data')
# Removing the unnecessary column
df.drop(columns=['Unnamed: 0'],inplace=True)
pd.set_option('display.max_colwidth', 100)
df.head()

We have 8094 speeches in the data


,Year,Session,Country,Transcript
0,2018,73,BRB,Let me begin by congratulating Ms. María Fernanda Espinosa Garcés on her election to preside ove...
1,2018,73,IND,"On my own behalf and on behalf of my country, India, I should like to begin by congratulating yo..."
2,2018,73,ARG,I would like to congratulate the President on her election to lead this session of the General A...
3,2018,73,JOR,It is an honour to take part in the general debate of the wonderful General Assembly once again....
4,2018,73,SWE,"Just a bit more than a week ago, we honoured the late former Secretary-General Kofi Annan in thi..."


**Pre-Processing**

Text Processing will be done in two levels:-
* Extracting meta text data and creating new features
* Process the text and create new features with tokens, cleaned text and stems.



In [4]:
# create features from meta text data
df['char_count'] = df['Transcript'].str.len()
df['words'] = df['Transcript'].str.split(' ')
df['sentences'] = df['Transcript'].str.split('.')
df['word_count'] = df['words'].str.len()
df['sentence_count'] = df['sentences'].str.len()
df['word_length'] = df['char_count'] / df['word_count']
df['sentence_length'] = df['word_count'] / df['sentence_count']

* Tokenize -  Tokenization is the task of chopping it up into pieces, called tokens , perhaps at the same time throwing away certain characters, such as punctuation.
* Clean - Stop Word Removal is the process of removing the stop words. A stop word is a commonly used word (such as “the”, “a”, “an”, “in”) that a search engine has been programmed to ignore, both when indexing entries for searching and when retrieving them as the result of a search queries.
* Stemmer - Stemming is the process of reducing inflected (or sometimes derived) words to their word stem, base or root form—generally a written word form.
* Lemmatizer - Lemmatization usually refers to doing things properly with the use of a vocabulary and morphological analysis of words, normally aiming to remove inflectional endings only and to return the base or dictionary form of a word, which is known as the lemma .

In [6]:
#create feature with text processing
df['Transcript'] = df['Transcript'].str.lower().map(lambda x: re.sub('\W+',' ', x))
df['Transcript'] = df['Transcript'].str.lower().map(lambda x: re.sub('united nations','united_nations', x))
df['token'] = df['Transcript'].apply(word_tokenize)

TypeError: expected string or bytes-like object

In [33]:
stop_words = set(stopwords.words('english'))
# I noticed that "'s" is not included in stopwords, while I think it doesn't bring much meaning in a text, so I'll add it to the set to remove from the cleaned tokens.
stop_words.add("'s")
stop_words.add("'")
stop_words.add("-")
stop_words.add("'")
debates['clean'] = debates['token'].apply(lambda x: [w for w in x if not w in stop_words and not w in punctuation])

import nltk
nltk.download('wordnet')
from nltk.corpus import wordnet as wn
def get_lemma(word):
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma

stemmer = nltk.stem.PorterStemmer()
debates['stems'] = [[format(get_lemma(token)) for token in speech] for speech in debates['clean']]
debates = debates.sort_values('year')


[nltk_data] Error loading wordnet: <urlopen error [Errno -3] Temporary
[nltk_data]     failure in name resolution>


In [34]:
debates['text'].str.find(sub='united nations')

174    -1
171    -1
172    -1
173    -1
175    -1
       ..
5478   -1
5477   -1
5476   -1
5485   -1
5548   -1
Name: text, Length: 7507, dtype: int64